In [1]:
# Install required packages
%pip install transformers datasets evaluate jiwer
%pip install librosa scikit-learn pandas
%pip install soundfile
%pip install tensorboard
%pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; 

In [2]:
import pandas as pd
import numpy as np
import torch
import librosa
from datasets import Dataset, Audio
from transformers import (
    WhisperFeatureExtractor, 
    WhisperTokenizer, 
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from jiwer import wer, cer, mer

2025-07-12 04:24:05.705437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752294246.048875      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752294246.144514      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ================================
# 1. DATA PREPARATION
# ================================

# Load the dataset from the TSV file
file_path = "/kaggle/input/noisy-data/noisy_data.tsv"
df = pd.read_csv(file_path, sep="\t", header=None, names=["ID", "Text"])

# Add full path to audio files
df["audio"] = df["ID"].apply(lambda x: f"/kaggle/input/noisy-audio-data-for-asr/noisy-audio-data-for-ASR/{x}")
df = df.rename(columns={"Text": "sentence"})

# Remove any rows with missing audio files or text
df = df.dropna()
df = df[df["sentence"].str.strip() != ""]

print(f"Dataset size: {len(df)} samples")
print(f"Sample data:\n{df.head()}")

# Train-validation split with stratification if needed
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# Save splits for reference
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

Dataset size: 4284 samples
Sample data:
                                ID  \
0  taf_02345_00348037167_noisy.wav   
1  taf_07049_00155837462_noisy.wav   
2  taf_09705_01218130267_noisy.wav   
3  taf_03219_00712757493_noisy.wav   
4  taf_00008_01305524612_noisy.wav   

                                            sentence  \
0  ஆஸ்த்ரேலியப் பெண்ணுக்கு முப்பத்தி மூன்று ஆண்டு...   
1            ஸ்ரீரங்கம் கோவிலில் வெடிகுண்டு மிரட்டல்   
2  உங்களுடைய உணவுக் கட்டுப்பாட்டைச் சொன்னால் மற்ற...   
3  ரஹானே மட்டும் ஆறுதலாக முப்பத்தி ஆறு ரன்கள் குவ...   
4               மனோரமாவிற்கு பூபதி என்ற மகன் உள்ளார்   

                                               audio  
0  /kaggle/input/noisy-audio-data-for-asr/noisy-a...  
1  /kaggle/input/noisy-audio-data-for-asr/noisy-a...  
2  /kaggle/input/noisy-audio-data-for-asr/noisy-a...  
3  /kaggle/input/noisy-audio-data-for-asr/noisy-a...  
4  /kaggle/input/noisy-audio-data-for-asr/noisy-a...  
Training samples: 3427
Validation samples: 857


In [4]:
# ================================
# 2. AUDIO PREPROCESSING & AUGMENTATION
# ================================

def verify_audio_file(audio_path):
    """Verify if audio file exists and is readable"""
    try:
        audio, sr = librosa.load(audio_path, sr=16000)
        return len(audio) > 0
    except:
        return False

def add_noise_augmentation(audio_array, noise_factor=0.005):
    """Add Gaussian noise for additional robustness"""
    noise = np.random.normal(0, noise_factor, audio_array.shape)
    return audio_array + noise

def normalize_audio(audio_array):
    """Normalize audio to prevent clipping"""
    max_val = np.max(np.abs(audio_array))
    if max_val > 0:
        return audio_array / max_val
    return audio_array

# Filter out invalid audio files
valid_train = []
valid_val = []

print("Validating audio files...")
for idx, row in train_df.iterrows():
    if verify_audio_file(row['audio']):
        valid_train.append(row)
    else:
        print(f"Invalid audio file: {row['audio']}")

for idx, row in val_df.iterrows():
    if verify_audio_file(row['audio']):
        valid_val.append(row)
    else:
        print(f"Invalid audio file: {row['audio']}")

train_df = pd.DataFrame(valid_train)
val_df = pd.DataFrame(valid_val)

print(f"Valid training samples: {len(train_df)}")
print(f"Valid validation samples: {len(val_df)}")

Validating audio files...
Valid training samples: 3427
Valid validation samples: 857


In [5]:
# ================================
# 3. WHISPER PROCESSOR SETUP
# ================================

# Initialize Whisper components for Tamil
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="ta", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="ta", task="transcribe")


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [6]:
# ================================
# 4. DATASET CREATION WITH NOISE HANDLING
# ================================

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Cast audio with target sampling rate
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(examples):
    """Prepare dataset with noise-robust preprocessing"""
    # Load and process audio
    audio = examples["audio"]
    audio_array = audio["array"]
    
    # Apply noise-robust preprocessing
    audio_array = normalize_audio(audio_array)
    
    # Optional: Add slight additional noise for robustness (uncomment if needed)
    # audio_array = add_noise_augmentation(audio_array, noise_factor=0.001)
    
    # Ensure audio is not too short or too long
    min_length = 1000  # ~0.06 seconds at 16kHz
    max_length = 480000  # ~30 seconds at 16kHz
    
    if len(audio_array) < min_length:
        # Pad short audio
        audio_array = np.pad(audio_array, (0, min_length - len(audio_array)), 'constant')
    elif len(audio_array) > max_length:
        # Truncate long audio
        audio_array = audio_array[:max_length]
    
    # Compute log-Mel input features
    examples["input_features"] = feature_extractor(
        audio_array, sampling_rate=16000
    ).input_features[0]
    
    # Clean up
    del examples["audio"]
    
    # Process text
    sentences = examples["sentence"]
    
    # Clean and normalize text
    if isinstance(sentences, str):
        sentences = sentences.strip()
    
    # Encode target text to label ids
    examples["labels"] = tokenizer(sentences).input_ids
    del examples["sentence"]
    
    return examples

# Apply preprocessing
print("Preprocessing training dataset...")
train_dataset = train_dataset.map(prepare_dataset, num_proc=1)

print("Preprocessing validation dataset...")
val_dataset = val_dataset.map(prepare_dataset, num_proc=1)

Preprocessing training dataset...


Map:   0%|          | 0/3427 [00:00<?, ? examples/s]

Preprocessing validation dataset...


Map:   0%|          | 0/857 [00:00<?, ? examples/s]

In [7]:
# ================================
# 5. DATA COLLATOR FOR NOISE-ROBUST TRAINING
# ================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Handle input features
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Handle labels
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # Remove BOS token if present
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [8]:
# ================================
# 6. METRICS FOR NOISE-ROBUST EVALUATION
# ================================

def compute_metrics(pred):
    """Compute comprehensive metrics for noisy ASR evaluation"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode token IDs to strings
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Calculate multiple metrics
    wer_score = wer(label_str, pred_str) * 100
    cer_score = cer(label_str, pred_str) * 100
    mer_score = mer(label_str, pred_str) * 100

    # Sentence Error Rate
    ser_score = (
        sum(ref.strip() != pred.strip() for ref, pred in zip(label_str, pred_str))
        / len(label_str)
    ) * 100

    return {
        "wer": wer_score,
        "cer": cer_score,
        "ter": mer_score,
        "ser": ser_score,
    }

In [9]:
# ================================
# 7. MODEL SETUP FOR NOISE-ROBUST TRAINING
# ================================
print("Loading pre-trained Whisper model...")
try:
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Freeze encoder for noise robustness (optional - uncomment if needed)
# for param in model.model.encoder.parameters():
#     param.requires_grad = False

# Verify model is loaded
print(f"Model type: {type(model)}")
print(f"Model config: {model.config}")
print(f"Model device: {next(model.parameters()).device}")

Loading pre-trained Whisper model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Model loaded successfully!
Model type: <class 'transformers.models.whisper.modeling_whisper.WhisperForConditionalGeneration'>
Model config: WhisperConfig {
  "_attn_implementation_autoset": true,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feat

In [17]:
# ================================
# 8. TRAINING ARGUMENTS FOR NOISY DATA
# ================================

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-base-ta-noisy-robust",
    eval_strategy="epoch",
    save_strategy="epoch",
    
    # Evaluation and saving
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    
    # Batch sizes - adjusted for noisy data
    per_device_train_batch_size=8,  # Reduced for stability
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,   # Compensate for baseer batch size
    
    # Learning rate - slightly lower for noisy data
    learning_rate=3e-5,
    warmup_steps=500,
    lr_scheduler_type="linear",
    
    # Memory optimization
    gradient_checkpointing=True,
    fp16=True,
    dataloader_pin_memory=False,
    
    # Training duration
    num_train_epochs=10,  # Reduced epochs for noisy data
    
    # Generation settings
    predict_with_generate=True,
    generation_max_length=225,
    
    # Logging
    logging_steps=50,
    report_to=["tensorboard"],
    
    # Additional stability settings
    max_grad_norm=1.0,
    weight_decay=0.01,
)

In [18]:
# # ================================
# # 9. TRAINER SETUP
# # ================================

# from transformers import WhisperForConditionalGeneration, WhisperConfig, EarlyStoppingCallback

# # Step 1: Load config and increase dropout
# config = WhisperConfig.from_pretrained("openai/whisper-base")
# config.dropout = 0.2

# # Step 2: Load model with new config
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base", config=config)

# # Step 3: Add EarlyStopping (optional but recommended)
# callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

# # Step 4: Use the model and training_args in your trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=processor.feature_extractor,
#     compute_metrics=compute_metrics,
#     callbacks=callbacks
# )


In [19]:
# ================================
# 9. TRAINER SETUP
# ================================
# Step 1: Load config and increase dropout
config = WhisperConfig.from_pretrained("openai/whisper-base")
config.dropout = 0.2

# Step 2: Load model with new config
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base", config=config)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


/tmp/ipykernel_35/2475419867.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [20]:
# # ================================
# # 10. TRAINING EXECUTION
# # ================================

# print("Starting noise-robust ASR training...")
# print(f"Training on {len(train_dataset)} noisy samples")
# print(f"Validating on {len(val_dataset)} noisy samples")

# # Train the model
# trainer.train(resume_from_checkpoint="/kaggle/input/last-checkpoint-001")

In [21]:
# ================================
# 10. TRAINING EXECUTION
# ================================

print("Starting noise-robust ASR training...")
print(f"Training on {len(train_dataset)} noisy samples")
print(f"Validating on {len(val_dataset)} noisy samples")

# Train the model
trainer.train()



Starting noise-robust ASR training...
Training on 3427 noisy samples
Validating on 857 noisy samples


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer,Cer,Ter,Ser
1,4.586600,1.886346,123.027211,317.691837,97.231183,100.000000
2,2.913500,2.571924,567.278912,385.255528,99.922114,100.000000
3,2.606300,2.480611,137.959184,392.714788,99.987674,100.000000
4,2.438800,2.357719,103.826531,77.974141,100.000000,100.000000
5,2.299400,2.184037,101.020408,79.144671,100.000000,100.000000
6,2.211700,2.122048,100.323129,79.114069,100.000000,100.000000
7,2.128600,2.060203,103.435374,87.407237,99.950698,100.000000
8,2.087100,2.029236,125.357143,102.507459,99.959317,100.000000
9,2.073800,2.018514,126.547619,108.909035,99.973129,100.000000


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183

TrainOutput(global_step=972, training_loss=2.918557575210132, metrics={'train_runtime': 11530.1485, 'train_samples_per_second': 2.972, 'train_steps_per_second': 0.093, 'total_flos': 2.00047686156288e+18, 'train_loss': 2.918557575210132, 'epoch': 9.0})

In [ ]:
# Save the final model
trainer.save_model("./whisper-base-ta-noisy-robust-final")
processor.save_pretrained("./whisper-base-ta-noisy-robust-final")

print("Training completed!")
print("Model saved to ./whisper-base-ta-noisy-robust-final")

In [ ]:
# ================================
# 12. CREATE DOWNLOADABLE ARCHIVES FOR KAGGLE
# ================================

import os
import shutil
import zipfile
from IPython.display import FileLink

def create_downloadable_archive(source_dir, archive_name):
    """Create a downloadable zip archive"""
    if os.path.exists(source_dir):
        # Create zip file
        shutil.make_archive(archive_name, 'zip', source_dir)
        zip_path = f"{archive_name}.zip"
        
        if os.path.exists(zip_path):
            file_size = os.path.getsize(zip_path) / (1024 * 1024)  # Size in MB
            print(f"✅ Created {zip_path} ({file_size:.2f} MB)")
            return zip_path
        else:
            print(f"❌ Failed to create {zip_path}")
            return None
    else:
        print(f"❌ Source directory {source_dir} does not exist")
        return None

# Create downloadable archives
print("\n🔄 Creating downloadable model archives...")

# 1. Final trained model
final_model_zip = create_downloadable_archive(
    "./whisper-base-ta-noisy-robust-final", 
    "whisper-base-ta-noisy-robust-final"
)

# 2. Last checkpoint from training
checkpoint_dir = "./whisper-base-ta-noisy-robust"
if os.path.exists(checkpoint_dir):
    # Find the last checkpoint
    checkpoints = [d for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
    if checkpoints:
        # Sort by checkpoint number
        checkpoints.sort(key=lambda x: int(x.split("-")[1]))
        last_checkpoint = checkpoints[-1]
        last_checkpoint_path = os.path.join(checkpoint_dir, last_checkpoint)
        
        print(f"📂 Found last checkpoint: {last_checkpoint}")
        
        # Create archive for last checkpoint
        last_checkpoint_zip = create_downloadable_archive(
            last_checkpoint_path, 
            f"whisper-base-ta-noisy-robust-{last_checkpoint}"
        )
    else:
        print("❌ No checkpoints found in training directory")
        last_checkpoint_zip = None
else:
    print("❌ Training directory does not exist")
    last_checkpoint_zip = None

# 3. Training logs and metrics
if os.path.exists("./whisper-base-ta-noisy-robust/runs"):
    tensorboard_logs_zip = create_downloadable_archive(
        "./whisper-base-ta-noisy-robust/runs", 
        "whisper-base-ta-noisy-robust-tensorboard-logs"
    )
else:
    tensorboard_logs_zip = None

# Create a comprehensive package with all files
print("\n📦 Creating comprehensive model package...")
comprehensive_package = "whisper-base-ta-noisy-robust-complete"
os.makedirs(comprehensive_package, exist_ok=True)

# Copy final model
if os.path.exists("./whisper-base-ta-noisy-robust-final"):
    shutil.copytree("./whisper-base-ta-noisy-robust-final", 
                    f"{comprehensive_package}/final_model", 
                    dirs_exist_ok=True)

# Copy last checkpoint
if os.path.exists(last_checkpoint_path):
    shutil.copytree(last_checkpoint_path, 
                    f"{comprehensive_package}/last_checkpoint", 
                    dirs_exist_ok=True)

# Copy training data splits
if os.path.exists("train.csv"):
    shutil.copy("train.csv", f"{comprehensive_package}/train.csv")
if os.path.exists("val.csv"):
    shutil.copy("val.csv", f"{comprehensive_package}/val.csv")

# Create README file
readme_content = f"""# Whisper base Tamil Noisy-Robust ASR Model

## Training Information
- base Model: openai/whisper-base
- Language: Tamil (ta)
- Task: Transcription
- Dataset: Noisy audio data for ASR
- Training Strategy: Noise-robust fine-tuning

## Model Files
- `final_model/`: Complete trained model ready for inference
- `last_checkpoint/`: Last training checkpoint
- `train.csv`: Training data split
- `val.csv`: Validation data split

## Usage
```python
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Load the model
model = WhisperForConditionalGeneration.from_pretrained("./final_model")
processor = WhisperProcessor.from_pretrained("./final_model")

# Use for inference
# (Add your inference code here)
```

## Training Configuration
- Batch Size: 16 (per device)
- Learning Rate: 1e-5
- Epochs: 15
- Early Stopping: 3 epochs patience
- Optimizer: AdamW with weight decay

## Evaluation Metrics
- WER (Word Error Rate)
- CER (Character Error Rate)
- SER (Sentence Error Rate)
- TER (Token Error Rate)

Training completed successfully!
"""

with open(f"{comprehensive_package}/README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

# Create comprehensive zip
comprehensive_zip = create_downloadable_archive(
    comprehensive_package, 
    "whisper-base-ta-noisy-robust-complete"
)

In [ ]:
# ================================
# 13. DISPLAY DOWNLOAD LINKS
# ================================

print("\n" + "="*60)
print("📥 DOWNLOAD LINKS FOR KAGGLE")
print("="*60)

download_links = []

if final_model_zip:
    download_links.append(("Final Trained Model", final_model_zip))
    print(f"🎯 Final Model: {final_model_zip}")

if last_checkpoint_zip:
    download_links.append(("Last Checkpoint", last_checkpoint_zip))
    print(f"📍 Last Checkpoint: {last_checkpoint_zip}")

if tensorboard_logs_zip:
    download_links.append(("TensorBoard Logs", tensorboard_logs_zip))
    print(f"📊 TensorBoard Logs: {tensorboard_logs_zip}")

if comprehensive_zip:
    download_links.append(("Complete Package", comprehensive_zip))
    print(f"📦 Complete Package: {comprehensive_zip}")

print("\n" + "="*60)
print("🔗 DIRECT DOWNLOAD COMMANDS")
print("="*60)

# Create download commands for each file
for name, file_path in download_links:
    if file_path and os.path.exists(file_path):
        print(f"\n# Download {name}")
        print(f"# File: {file_path}")
        print(f"# Size: {os.path.getsize(file_path) / (1024*1024):.2f} MB")
        
        # For Kaggle, files in the working directory are automatically available for download
        # Just need to display the file link
        display(FileLink(file_path))

print("\n" + "="*60)
print("✅ All model files are ready for download!")
print("📝 Click the links above to download the files")
print("💡 The 'Complete Package' contains everything you need")
print("="*60)